# Dependencies

Install dependencies with the following...

    # pip or pip3 depending on setup
    pip3 install plotly statsmodels requests pandas

# Acquire Data

Pulls data from the URL used by the dashboard on the [coronavirus.maryland.gov](https://coronavirus.maryland.gov/) website.  Should dump an HTTP status (looking for a 200) and last date present in data.

In [ ]:
import requests
import pandas
import json
import plotly.express as px
import plotly.graph_objects as go

pandas.options.display.max_rows = None

params = {
    "f": "json",
    "where": "1=1",
    "returnGeometry": "false",
    "spatialRel": "esriSpatialRelIntersects",
    "outFields": "*",
    "orderByFields": "ReportDate asc",
    "resultOffset": "0",
    "resultRecordCount": "32000",
    "resultType": "standard",
    "cacheHint": "true"
}

URL = 'https://services.arcgis.com/njFNhDsUCentVYJW/arcgis/rest/services/MASTER_CaseTracker_3/FeatureServer/0/query'
            
response = requests.request("GET", URL, params=params)

# 200 status means we're good to go
print("HTTP Status: {}".format(response.status_code))

# Dump the last date
print("Last Data: {}".format(pandas.to_datetime(response.json()['features'][-1]['attributes']['ReportDate'], unit='ms')))

# Top-Level Keys

Notable keys/values...
  * `fields` - semi-definition of each value (maybe makes sense in ESRI land?)
  * `features` - this is where the real daily data lies

In [ ]:
payload = response.json()

# pandas only used for easy display
display(pandas.DataFrame(payload.keys()))

# Keys in "Actual Data"

Once flattened, you can see what data is avaialble for each day.  Note that not all data is present for all days (e.g. shifts in binning of reporting over time will have gaps).

In [ ]:
# flatten a bit, as each item is rooted under an uncessary attributes key 
daily_numbers = [element['attributes'] for element in payload['features']]

# pandas only used for easy display
display(pandas.DataFrame(daily_numbers[0].keys()))

# DataFrame Time

Pull into Pandas data frame...

In [ ]:
# slurp into dataframe
df = pandas.read_json(json.dumps(daily_numbers), orient='records')

# Bed Usage Over Time

This regenerates the "ICU and Acute Hospital Beds for COVID-19, Currently in Use" chart from the MD page in Plotly.

In [ ]:
filtered = df.filter(items=['ReportDate', 'bedsAcute', 'bedsICU', 'bedsTotal'])
filtered['ReportDate'] = pandas.to_datetime(df['ReportDate'], unit='ms')

fig = go.Figure()
for x in ['bedsAcute', 'bedsICU', 'bedsTotal']:
    fig.add_trace(go.Scatter(x=filtered['ReportDate'], y=df[x], mode='lines+markers', name=x))
fig.show()

# Bed Usage Over Time (with Trendline)

Same as above, but with a [LOWESS](https://en.wikipedia.org/wiki/Local_regression) trendline (and a slightly modified x-axis).

In [ ]:
filtered = df.filter(items=['ReportDate', 'bedsAcute', 'bedsICU', 'bedsTotal'])

# for whatever reason lowess doesn't like datetimes?
# change to days from last data (last day = 0, day before = -1, 2 days before = -2, ...)

# filtered['ReportDate'] = pandas.to_datetime(df['ReportDate'], unit='ms')
filtered['ReportDate'] = [i - len(filtered['ReportDate']) + 1 for i in range(len(filtered['ReportDate']))]

df_melt = filtered.melt(id_vars='ReportDate', value_vars=['bedsAcute', 'bedsICU', 'bedsTotal'])
fig = px.scatter(df_melt, x='ReportDate' , y='value', color='variable', trendline="lowess")
fig.show()

# Hospitalized Data

Not always clear how/if this data correspondes to `bedsAcute`, `bedsICU`, and `bedsTotal`.

In [ ]:
# tabluar dump
filtered = df.filter(items=['ReportDate', 'total_hospitalized', 'hospitalizedDelta'])
filtered['ReportDate'] = pandas.to_datetime(df['ReportDate'], unit='ms')
display(filtered)

In [ ]:
# line chart
fig = go.Figure()
for x in ['hospitalizedDelta']:
    fig.add_trace(go.Scatter(x=filtered['ReportDate'], y=filtered[x], mode='lines+markers', name=x))
fig.show()

# Export to CSV

For all your excel needs ; )

In [ ]:
# Dump raw data in less verbose CSV
df.to_csv('md-data.csv')